# Спасибо ребятам из команды Sokolov и Golobokov за идею оформления ноутбука

# 1. Загрузка необходимых библиотек

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, OrdinalEncoder, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score, roc_curve, \
                            plot_confusion_matrix, precision_recall_curve, auc, plot_precision_recall_curve, average_precision_score
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from math import log as log
import os

# Загрузка данных Kaggle
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
PATH_to_file = '/kaggle/input/sf-dst-scoring/'

# Загружаем дополнительные функции

# Plot Boxplots
def plot_outliers(df, width=20, height=10):
    '''
    df - Pandas DataFrame
    width - Figure width
    height - Figure height
    '''
    fig = plt.figure(figsize=(width, height))
    sns.boxplot(x="variable", y="value", data=pd.melt(df))
    plt.show()
    

# Plot BoxPlots compare to target class
def plot_boxplots(df, columns, target, nrows=2, ncols=2, figsize=(15,15)):
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    plt.subplots_adjust(wspace = 0.5)
    axes = axes.flatten()
    for i in range(len(columns)):
        sns.boxplot(x="default", y=columns[i], data=df, orient = 'v', ax=axes[i], showfliers=True)
        
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5, use_abs=True, target=None, filter_target=False):
    '''
    df - Pandas DataFrame
    n - number of return values
    use_abs - use absolute numbers
    target - feature name to return
    filter_target - use feature filter
    '''
    
    if use_abs:
        au_corr = df.corr().abs().unstack()
    else:
        au_corr = df.corr().unstack()
        
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    
    if filter_target and target is not None:
        return au_corr.filter(like=target, axis=0)[0:n]
    
    return au_corr[0:n]


# Посчитаем важность признаков при помощи однофакторного дисперсионного анализа (ANOVA)
def plot_fclassif(df, columns, target):
    '''
    sklean.feature_selection.f_classif wrapper
    ------------------------------------------
    df - Pandas DataFrame
    columns - numerical features
    target - Our target feature
    '''
    
    if df is not None and columns is not None and target is not None:
        imp_num = pd.Series(f_classif(df[columns], df[target])[0], index = columns)
        imp_num.sort_values(inplace = True)
        imp_num.plot(kind = 'barh')
    else:
        raise Exception('Fill params!')


def plot_mutual_info_classif(df, columns, target):
    '''
    sklean.feature_selection.mutual_info_classif
    ------------------------------------------
    df - Pandas DataFrame
    columns - binary, categorical features
    target - Our target feature
    '''
    if df is not None and columns is not None and target is not None:
        imp_cat = pd.Series(mutual_info_classif(df[columns], df[target],
                                            discrete_features =True), index = columns)
        imp_cat.sort_values(inplace = True)
        imp_cat.plot(kind = 'barh')
    else:
        raise Exception('Fill params!')
        
# Отрисовать ROC кривую
def roc_auc_plot(y_true, y_pred_proba):
    '''
    Функция считает AUC и отрисовывает ROC кривую:
        y_true - истинное значение класса
        y_pred_proba - предсказанная вероятность класса [:, 1]
    '''
    # Посчитать значения ROC кривой и значение площади под кривой AUC
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    
    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.3f}')
    plt.title('ROC AUC = %0.3f' % roc_auc, fontsize=15)
    plt.xlabel('False positive rate (FPR)', fontsize=15)
    plt.ylabel('True positive rate (TPR)', fontsize=15)
    plt.legend(fontsize=15, loc = 'lower right')

# Выводим распределение переменной и ее логарифма
def plot_dist_log(df, column, nrows=1, ncols=2, figsize=(15,15)):
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    plt.subplots_adjust(wspace = 0.5)
    axes = axes.flatten()
    axes[0].title.set_text(column)
    axes[1].title.set_text('np.log('+column+')')
    sns.distplot(df[column], ax=axes[0], axlabel=False, kde=False)
    sns.distplot(np.log(df[column] + 1), ax=axes[1], axlabel=False, kde=False)
    fig.suptitle(f'Распределение признака {column} и np.log({column})', fontsize=16, y=1.12)
    
    # Выводим дополнительную информацию о признаке
    feature_info = dict(count=df[column].count(), nan=df[column].isnull().sum(), 
                    min=df[column].min(), max=df[column].max(), median=df[column].median(), mean=df[column].mean())

    fi = pd.Series(feature_info)
    p = pd.DataFrame(fi, columns=[column])

    display(p)

# Выводим метрики модели
def show_model_metrics(y_true, y_pred, y_probs):
    print(f'f1_score: {f1_score(y_true, y_pred)}')
    print(f'precision_score: {precision_score(y_true, y_pred)}')
    print(f'recall_score: {recall_score(y_true, y_pred)}')
    print(f'roc_auc: {roc_auc_score(y_true, y_probs)}')
    
    
# Выводим PRC-AUC
def plot_prc_auc(model, X, y, y_prob):
    average_precision = average_precision_score(y, y_prob)
    disp = plot_precision_recall_curve(model, X, y)
    disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))
    
# Нарисовать кросс-валидацию
def plot_crossval_score(cross_val, metric, figsize=(20, 10)):
    plt.figure(figsize=figsize)
    sns.lineplot(data=cross_val, x=range(len(cross_val['score_time'])), y="test_score", legend='full', label='тестовая выборка')
    sns.lineplot(data=cross_val, x=range(len(cross_val['score_time'])), y="train_score", legend='full', label='тренировочная выборка')
    sns.lineplot(data=cross_val, x=range(len(cross_val['score_time'])), y=np.mean(cross_val['test_score']), label='среднее значение на тесте')
    sns.lineplot(data=cross_val, x=range(len(cross_val['score_time'])), y=np.mean(cross_val['train_score']), legend='full', label='среднее значение на трейне')
    plt.xlabel("Номер фолда")
    plt.ylabel(metric)
    plt.title(f"Кросс-валидация по ROC-AUC (KFold={len(cross_val['score_time'])})")

def prepare_data(df):
    # Готовим данные для обучения модели
    train_data = df[df.Train == 1].drop(['Train'], axis=1)
    test_data = df[df.Train == 0].drop(['Train'], axis=1)

    y = train_data.default.values
    X = train_data.drop(['default'], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
    
    return X_train, X_test, y_train, y_test


# Удаляем выбросы в датасете 
def remove_outliers_irq_dataset(df, iqr_detect=1.5):
    '''
    df - Pandas DataFrame
    iqr_detect - IQR to detect and filter outliers. Default: 1.5 
    '''
    
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1

    return df[~((df < (Q1 - iqr_detect * IQR)) |(df > (Q3 + iqr_detect * IQR))).any(axis=1)]

# Удаляем выбросы по конкретному признаку
def remove_outliers_iqr_column(df_in, col_name):
    '''
    df_in - Pandas DataFrame
    col_name - DataFrame column to detect outliers
    '''
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    
    return df_out

# Загузка данных с обработкой признаков
def load_data(fill_na='popular', preprocess=False, drop_outliers=False, outliers_column=None, log_numerical=True):
    '''load_data - загружает данных и возвращает обработанный Pandas DataFrame
        fill_na - как заполнять пропущенные значения [mean, mode, drop]
        preprocess - выполнять или нет ручную обработку данных [False если используется pipeline]
        drop_outliers - удалять выбросы или нет 
        log_numerical - логарифмировать числовые признаки или нет
    '''
    df_train = pd.read_csv(PATH_to_file+'train.csv')
    df_test = pd.read_csv(PATH_to_file+'test.csv')
    
    df_train['Train'] = 1 # трэйн
    df_test['Train'] = 0 # тест

    df = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем
    
    # внесем данные из резюме в списки 
    time_cols = ['app_date']
    # бинарные переменные
    bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport', 'default_peak', 'normal_age']
    # категориальные переменные
    cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
    # числовые переменные
    num_cols = ['client_id', 'age','decline_app_cnt','score_bki','bki_request_cnt','income','days_between_firstdate', 'mean_region_bki'] # days_between_firstdate - признак добавим позже, как разницу между
       
    # Логарифмируем переменные
    if log_numerical:
        df['age'] = np.log(df['age'] + 1)
        df['decline_app_cnt'] = np.log(df['decline_app_cnt'] + 1)
        df['bki_request_cnt'] = np.log(df['bki_request_cnt'] + 1)
        df['income'] = np.log(df['income'] + 1)
    
    # Дата
    df.app_date = pd.to_datetime(df.app_date, format='%d%b%Y')
    
    # Посмотрим на максимальное и минимальное значение
    start_date = df.app_date.min()
    end_date = df.app_date.max()
    
    # Создадим новый признак - номер дня от минимальной даты
    df['days_between_firstdate'] = (df.app_date - start_date).dt.days.astype('int')
    
    # Признак, когда дефолт идет на спад
    df['default_peak'] = (df.days_between_firstdate > 90).astype('int')
    
    # Средний score_bki по региону
#     mean_score_bki = df.groupby('region_rating')['score_bki'].mean().to_dict()
#     df['mean_region_bki'] = df.apply(lambda x: x.score_bki / mean_score_bki[x.region_rating], axis=1)
    
    # Обычно благоприятный возраст для кредита 30-40 лет
    df['normal_age'] = ((df.age >= 30) & (df.age <= 40)).astype('int')
    
    # Ручная обработка данных
    if preprocess:
        labelEncoder = LabelEncoder()

        # Закодируем бинарные признаки
        for column in bin_cols:
            df[column] = labelEncoder.fit_transform(df[column])

        # Закодируем признак education, но сперва заполним пропуски самым частым значением
        # Заполним пропуски
        if fill_na in ['mean', 'mode', 'drop']:
            if fill_na == 'mean':
                df.education = df.education.fillna(df.education.value_counts()
                                [df.education.value_counts() == int(df.education.value_counts().median())].index[0])
            elif fill_na == 'mode':
                df.education = df.education.fillna(df.education.mode()[0])
            elif fill_na == 'drop':
                df = df.dropna(axis=0)
        else:
            df.education = df.education.fillna('SCH')

        # Закодируем признак образования
        df.education = labelEncoder.fit_transform(df.education)

        # Категориальные признаки в дамми переменные
        df = pd.get_dummies(df, columns=cat_cols)
        
    df.drop('app_date', axis=1, inplace=True)
    
    # Удаляем выбросы
    if drop_outliers and outliers_column is not None:
        return remove_outliers_iqr_column(df, outliers_column)
    
    return df   

# Пайплайн обработки данных
def prepare_data_pipeline(scaler='standard', polynomial_features=False, polynomial_n=2):
    # бинарные переменные
    bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport', 'default_peak', 'normal_age']
    # категориальные переменные
    cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
    # числовые переменные
    num_cols = ['client_id', 'age','decline_app_cnt','score_bki','bki_request_cnt','income','days_between_firstdate'] # days_between_firstdate - признак добавим позже, как разницу между
    # Фичи для полиномизации
    poly_cols = ['decline_app_cnt', 'score_bki', 'bki_request_cnt']
    
    bin_transformer = make_pipeline(OrdinalEncoder())
    cat_transformer = make_pipeline(SimpleImputer(strategy='most_frequent', missing_values=np.nan), OrdinalEncoder(), OneHotEncoder())
    poly_transformer = make_pipeline(PolynomialFeatures(polynomial_n))
    
    if scaler == 'standard':
        num_transformer = make_pipeline(StandardScaler())
    elif scaler == 'robust':
        num_transformer = make_pipeline(RobustScaler())
    elif scaler == 'minmax':
        num_transformer = make_pipeline(MinMaxScaler())
    else:
        num_transformer = make_pipeline(StandardScaler())
        
    if polynomial_features:
        transformers = [
                        ('bin', bin_transformer, bin_cols),
                        ('cat', cat_transformer, cat_cols),
                        ('num', num_transformer, num_cols),
                        ('poly', poly_transformer, poly_cols)
        ]
    else:
        transformers = [
                        ('bin', bin_transformer, bin_cols),
                        ('cat', cat_transformer, cat_cols),
                        ('num', num_transformer, num_cols),                    
        ]
    
    return ColumnTransformer(transformers=transformers, remainder='passthrough')



from sklearn.base import BaseEstimator, TransformerMixin

# Генератор стандартных фич
class FeatureGenerator(BaseEstimator, TransformerMixin):

    def __init__(self, feature_list = None, primitives=['mul']):
        self.feature_list = feature_list
        self.used_features = []
        self.primitives = primitives

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        X_ = X.copy() # работаем с копией, чтобы не изменять оригинальный датасет
        self.used_features = []  # обнуляем список использованных фич перед каждым вызовом transform

        if self.feature_list is None:
            self.feature_list = X_.columns

        return self.feature_generator(X_)
    
    def feature_generator(self, X):
        # Перебираем наши признаки и генерируем новые
        for i, feature in enumerate(self.feature_list):
            # Добавяем признак в список использованных
            self.used_features.append(feature)
            # Генерируем новые признаки по списку оставшихся
            for feature_ in list(set(self.feature_list) - set(self.used_features)):
                if self.primitives == 'all':
                    self.primitives = ['mul', 'sum', 'sub', 'div', 'mean', 'std', 'median']
                for primitive in self.primitives:
                    feature_name = f'{feature}_{primitive}_{feature_}'
                    if primitive == 'mul':
                        X[feature_name] = X[feature] * X[feature_]
                    if primitive == 'sum':
                        X[feature_name] = X[feature] + X[feature_]
                    if primitive == 'sub':
                        X[feature_name] = X[feature] - X[feature_]
                    if primitive == 'div':
                        X[feature_name] = X[feature] / (X[feature_]+1)
                    if primitive == 'mean':
                        X[feature_name] = X[feature] / (X[feature_].mean() + 1)
                    if primitive == 'std':
                        X[feature_name] = X[feature] / (X[feature_].std() + 1)
                    if primitive == 'median':
                        X[feature_name] = X[feature] / (X[feature_].median() + 1)
                    
        return X
    

/kaggle/input/sf-dst-scoring/train.csv
/kaggle/input/sf-dst-scoring/test.csv
/kaggle/input/sf-dst-scoring/sample_submission.csv


In [2]:
RANDOM_SEED = 42
!pip freeze > requirements.txt
CURRENT_DATE = pd.to_datetime('26/10/2020')

# 2. Загрузка данных

In [ ]:
df_train = pd.read_csv(PATH_to_file+'train.csv')
df_test = pd.read_csv(PATH_to_file+'test.csv')
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', df_train.shape)
display(df_train.head())
print('Размерность тестового датасета: ', df_test.shape)
display(df_test.head())

In [ ]:
# Выполняем объединение датасетов в один
df_train['Train'] = 1 # трэйн
df_test['Train'] = 0 # тест

df = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем

In [ ]:
df.head()

# 3. Выполняем EDA

Описания полей датасета

* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия “хорошей” работы
* bki_request_cnt - количество запросов в БКИ
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - связь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - флаг дефолта по кредиту

In [ ]:
df.info()

In [ ]:
# Посмотрим дополнительную информацию по датасету

print(f'Размер обучающей выборки: {df_train.shape}, размер тестовой выборки: {df_test.shape}')

# Количество уникальных значений
for column in df.columns:
    print(f'{column} = {len(df[column].unique())}')

По данным видим, что у нас имеется 1 признак - временной, 5 бинарных, 6 категориальных и 6 числовых.

In [ ]:
# внесем данные из резюме в списки 
time_cols = ['app_date']
# бинарные переменные
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
# категориальные переменные
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
# числовые переменные
num_cols = ['client_id', 'age','decline_app_cnt','score_bki','bki_request_cnt','income','days_between_firstdate'] # days_between_firstdate - признак добавим позже, как разницу между минимальным app_date и app_date 

In [ ]:
# Посмотрим на пропуски
import missingno as msno
msno.matrix(df)

In [ ]:
# Посмотрим на нашу целевую переменную
sns.countplot(x='default', data=df)
display(df.default.value_counts())

Видим,что у нас сильно несбалансированная выборка, что может привести к неудовлетворительным результатам при обучении логистической регрессии.

# 4 Посмотрим на признаки

## age

In [ ]:
plot_dist_log(df[df.Train==1], 'age', figsize=(8,4))

**age** - берем логарифмированный признак в работу

In [ ]:
df['age'] = np.log(df['age'] + 1)

## decline_app_cnt

In [ ]:
#'age','decline_app_cnt','score_bki','bki_request_cnt','income'
plot_dist_log(df[df.Train==1], 'decline_app_cnt', figsize=(8,4))


**decline_app_cnt**: логарифмирование особо не помогло, тем не менее оставим его в модели.

In [ ]:
df['decline_app_cnt'] = np.log(df['decline_app_cnt'] + 1)

## score_bki

In [ ]:
plot_dist_log(df[df.Train==1], 'score_bki', figsize=(8,4))

**score_bki** распределение изначально нормальное. Логарифмирование смещает распределение вправо. Оставим признак как есть.

## bki_request_cnt

In [ ]:
plot_dist_log(df[df.Train==1], 'bki_request_cnt', figsize=(8,4))

**bki_request_cnt** оставим логарифмированное распределение.

In [ ]:
df['bki_request_cnt'] = np.log(df['bki_request_cnt'] + 1)

## income

In [ ]:
plot_dist_log(df[df.Train==1], 'income', figsize=(8,4))

**income** логарифмированное распределение стало похоже на нормальное. Оставим в этом виде.

In [ ]:
df['income'] = np.log(df['income'] + 1)

## Категориальные признаки
Посмотрим на 'education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time'

## education

In [ ]:
sns.countplot(x='education', data=df[df.Train==1], hue='education')

## region_rating

In [ ]:
plot_dist_log(df[df.Train==1], 'region_rating', figsize=(8,4))

**region_rating** распределение похоже на нормальное. Посмотрим как модель поведет себя при обучение и примем решение о логарифмировании.

## home_address

In [ ]:
plot_dist_log(df[df.Train==1], 'home_address', figsize=(8,4))

**home_address** имеет 3 категории. Оставляем как есть.

## work_address

In [ ]:
plot_dist_log(df[df.Train==1], 'work_address', figsize=(8,4))

**work_address** аналогично home_address

## sna

In [ ]:
plot_dist_log(df[df.Train==1], 'sna', figsize=(8,4))

**sna** видим, что распределение смещено вправо. Логарифмирование не вносит значительных изменений. Оставим как есть.  

## first_time

In [ ]:
plot_dist_log(df[df.Train==1], 'first_time', figsize=(8,4))

**first_time** нормальное распределение смещено влево. Оставляем как есть.

## app_date

In [ ]:
# Преобразуем признак в дату
df.app_date = pd.to_datetime(df.app_date, format='%d%b%Y')

In [ ]:
# Посмотрим на максимальное и минимальное значение
start_date = df.app_date.min()
end_date = df.app_date.max()

print(start_date, end_date)

In [ ]:
# Создадим новый признак - номер дня от минимальной даты
df['days_between_firstdate'] = (df.app_date - start_date).dt.days.astype('int')

Посмотрим как меняется распределение по дням в зависимости от дефолта

In [ ]:
train_df = df[(df.Train==1)]
sns.distplot(train_df[train_df.default==0].days_between_firstdate, hist=False, rug=True)
sns.distplot(train_df[train_df.default==1].days_between_firstdate, hist=False, rug=True)

Из графика видно, что примерно с 90 дня банк стал лучше справляться с несостоятельными клиентами. Попробуем из этого предположения создать новый бинарный признак.

In [ ]:
df['default_peak'] = (df.days_between_firstdate > 90).astype('int')

Посмотрим распределение дефолта по регионам

In [ ]:
train_df = df[(df.Train==1)]
sns.countplot(x="region_rating", hue="default", data=train_df)

## Бинарные признаки 'sex', 'car', 'car_type', 'good_work', 'foreign_passport'

Данные признаки содержат всего по два значения, поэтому в будущем просто закодируем их через LabelEncoder

## Посмотрим на корреляцию признаков

In [ ]:
sns.heatmap(train_df[num_cols+['default']].corr(), annot=True, fmt='.2')

Как видим особой корреляции между признакими нету, поэтому оставляем все в работе.

## Значимость переменных по ANOVA F-Test

In [ ]:
plot_fclassif(train_df, num_cols, 'default')

Из f-теста видим, что основной акцент при работе над новыми признаками следует уделить score_bki, decline_app_cnt и bki_request_cnt.

## Значимость категориальных переменных

In [ ]:
labelEncoder = LabelEncoder()

# Закодируем бинарные признаки
for column in bin_cols:
    df[column] = labelEncoder.fit_transform(df[column])
    
# Закодируем признак education, но сперва заполним пропуски самым частым значением
df.education = df.education.fillna('SCH')
df.education = labelEncoder.fit_transform(df.education)

In [ ]:
plot_mutual_info_classif(df[df.Train==1], bin_cols+cat_cols, 'default')

Видим, что сильная взаимная информация между таргетом и признаками sna, first_time, region_rating и home_address. Это знание поможет нам в feature engineering.

## Посмотрим на выбросы

In [ ]:
train_df = df[df.Train==1]
plot_outliers(train_df[num_cols[1:6]])

Видим, что присутствует достаточное количество выбросов в наших признаках. Попробуем обучить модель с ними, а дальше будет видно.

# 5. Обработка данных

In [ ]:
# Категориальные признаки
df = pd.get_dummies(df, columns=cat_cols)

In [ ]:
# Выполним нормализацию данных
rs = RobustScaler() # Используем робастный нормализатор, т.к. он устойчив к выбросам

df[num_cols] = rs.fit_transform(df[num_cols])

In [ ]:
# Посмотрим еще раз на наши данные
df.head()

In [ ]:
# Удаляем лишние признаки
df.drop(['app_date'], axis=1, inplace=True)

# 6. Построение модели

In [ ]:
# Готовим данные для обучения модели
train_data = df[df.Train == 1].drop(['Train'], axis=1)
test_data = df[df.Train == 0].drop(['Train'], axis=1)

y = train_data.default.values
X = train_data.drop(['default'], axis=1)

In [ ]:
# Разбиваем нашу выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

## Выполняем обучение модели

In [ ]:
model = LogisticRegression(random_state=RANDOM_SEED, max_iter=500)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:,1]


## Проводим оценку модели

In [ ]:
show_model_metrics(y_test, y_pred, y_pred_prob)

## ROC кривая

In [ ]:
roc_auc_plot(y_test, y_pred_prob)

## Матрица ошибок

In [ ]:
plot_confusion_matrix(model, X_test, y_test, values_format='d')

Как видим наша модель очень плохо предсказывает дефолтных клиентов из-за сильной несбалансированности нашей выборки. Хотя ROC-AUC показывает достаточно высокое значение, наша  модель выдает очень большую ошибку второго рода. С этим нужно что-то делать. По f1_score можно заметить, что модель несправляется с задачей. Посмотрим еще на одну метрику PRC-AUC, которая поможет оценить алгоритм на несбалансированных данных.

## Precision-Recall кривая

In [ ]:
plot_prc_auc(model, X_test, y_test, y_pred_prob)

## Кросс-валидация

In [ ]:
validate = cross_validate(model, X_test, y_test, cv=10, scoring='roc_auc', return_train_score=True)

In [ ]:
plot_crossval_score(validate, 'ROC-AUC')
print(np.mean(validate['test_score']))

Полученная модель показывает плохие результаты, хотя метрика ROC-AUC имеет достаточно высокое значение (0.745).
По матрице ошибок видно,что мы предсказали всего **43** дефолтных клиента из **1784**, что принесет банку убытки, потому что наша модель будет одобрять кредиты клиентам, которые не смогут их вернуть.
По PR-AUC видно, что качество нашей модели достаточно низкое.

Как видим, при кросс-валидаций на 10 фолдах ROC-AUC достаточно стабилен. Имеется небольшой провал на обучающей выборке. Далее попробуем подобрать параметры модели и посмотреть на результат.

## Поиск оптимальных параметров модели

In [ ]:
# Используем GridSearchCV для поиска оптимальных параметров модели
# model = LogisticRegression(random_state=RANDOM_SEED)

# iter_ = 500
# epsilon_stop = 1e-3
# C = np.arange(0, 1, 10)

# param_grid = [
#     {'penalty': ['l1'], 
#      'solver': ['liblinear', 'lbfgs'], 
#      'class_weight':['none', 'balanced'], 
#      'multi_class': ['auto','ovr'], 
#      'max_iter':[iter_],
#      'C': C,
#      'tol':[epsilon_stop]},
#     {'penalty': ['l2'], 
#      'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
#      'class_weight':['none', 'balanced'], 
#      'multi_class': ['auto','ovr'], 
#      'max_iter':[iter_],
#      'C': C,
#      'tol':[epsilon_stop]},
#     {'penalty': ['none'], 
#      'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
#      'class_weight':['none', 'balanced'], 
#      'multi_class': ['auto','ovr'], 
#      'max_iter':[iter_],
#      'tol':[epsilon_stop]},
# ]
# gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
# gridsearch.fit(X_train, y_train)
# model = gridsearch.best_estimator_
# # Выведем параметры
# best_parameters = model.get_params()
# for param_name in sorted(best_parameters.keys()):
#         print('\t%s: %r' % (param_name, best_parameters[param_name]))
# # Выведем метрики
# preds = model.predict(X_test)
# print('Accuracy: %.4f' % accuracy_score(y_test, preds))
# print('Precision: %.4f' % precision_score(y_test, preds))
# print('Recall: %.4f' % recall_score(y_test, preds))
# print('F1: %.4f' % f1_score(y_test, preds))

На подобранных параметрах непохо подросла метрика f1_score. Попробуем обучить модель с новыми параметрами.

## Вторая модель с подобранными параметрами

In [ ]:
X_train, X_test, y_train, y_test = prepare_data(df)

model = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None, 
                           multi_class= 'auto', 
                           n_jobs= None, 
                           penalty= 'none', 
                           solver = 'sag',
                           max_iter = 500,
                           verbose= 0, 
                           warm_start= False)

model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

## Оценим качество второй модели

In [ ]:
show_model_metrics(y_test, y_pred, y_pred_prob)
validate = cross_validate(model, X_test, y_test, cv=10, scoring='roc_auc', return_train_score=True)
print(np.mean(validate['test_score']))

Посмотрим на матрицу ошибок.

In [ ]:
plot_confusion_matrix(model, X_test, y_test, values_format='d')

In [ ]:
roc_auc_plot(y_test, y_pred_prob)

In [ ]:
plot_prc_auc(model, X_test, y_test, y_pred_prob)

# Попробуем разные Pipeline

# preprocessing->Different classifiers

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

df = load_data(preprocess=False, drop_outliers=False, outliers_column='income', log_numerical=True)

# Разобьем данные
X_train, X_test, y_train, y_test = prepare_data(df)

# Модель 1
lr0 = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None, 
                           multi_class= 'auto', 
                           n_jobs= None, 
                           penalty= 'none', 
                           solver = 'sag',
                           max_iter = 500,
                           verbose= 0, 
                           warm_start= False)

# Модель 2
lr1 = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1.5, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None, 
                           multi_class= 'auto', 
                           n_jobs= None, 
                           penalty= 'l2', 
                           solver = 'lbfgs',
                           max_iter = 500,
                           verbose= 0, 
                           warm_start= False)

# Предобработка данных
preprocessing = prepare_data_pipeline(scaler='robust', polynomial_features=False, polynomial_n=3)

# # Воспользуемся RandomizedSearchCV для более быстрого подбора параметров модели
# # Количество деревьев
# n_estimators = np.arange(200, 1000, 200)
# # Количество фич
# max_features = ['auto', 'sqrt']
# # Глубина дерева
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Минимальное количество сэмплов для сплита
# min_samples_split = [2, 5, 10]
# # Минимальное количество сэмплов в листе
# min_samples_leaf = [1, 2, 4]
# # Метод выбора сэмплов
# bootstrap = [True, False]

# # Рандомная сетка подбора параметров для деревьев
# random_grid_tree = {
#                        'max_features': max_features,
#                        'max_depth': max_depth,
#                        'min_samples_split': min_samples_split,
#                        'min_samples_leaf': min_samples_leaf,
#                        'bootstrap': bootstrap}

# parameters = { 'tree': {
#                             'decisiontreeclassifier__max_features': max_features,
#                             'decisiontreeclassifier__min_samples_split': min_samples_split,
#                             'decisiontreeclassifier__min_samples_leaf': min_samples_leaf,
#                             'decisiontreeclassifier__max_depth': max_depth,
#                         },
#                 'forest': {
#                             'randomforestclassifier__max_features': max_features,
#                             'randomforestclassifier__min_samples_split': min_samples_split,
#                             'randomforestclassifier__min_samples_leaf': min_samples_leaf,
#                             'randomforestclassifier__bootstrap': bootstrap,
#                             'randomforestclassifier__max_depth': max_depth,
#                             #'randomforestclassifier__n_estimators': n_estimators,
#                         },
#                 'gbc':  {
#                             'gradientboostingclassifier__max_features': max_features,
#                             'gradientboostingclassifier__min_samples_split': min_samples_split,
#                             'gradientboostingclassifier__min_samples_leaf': min_samples_leaf,
#                             'gradientboostingclassifier__max_depth': max_depth,
#                             #'gradientboostingclassifier__n_estimators': n_estimators,
#                         },
#                  'xgb': {
#                             'xgbclassifier__min_child_weight': [1, 5, 10],
#                             'xgbclassifier__gamma': [0.5, 1, 1.5, 2, 5],
#                             'xgbclassifier__subsample': [0.6, 0.8, 1.0],
#                             'xgbclassifier__colsample_bytree': [0.6, 0.8, 1.0],
#                             'xgbclassifier__max_depth': [3, 4, 5, 10]
#                  },
#                  'ada': {
#                              'adaboostclassifier__n_estimators':[500, 1000, 1500, 2000], 
#                              'adaboostclassifier__learning_rate':[0.05, 0.1, 0.15, 0.2]
#                  }
# }


# # Подбор параметров для Tree-based алгоритмов
# tree_classifiers = [
#     #{ 'classifier': DecisionTreeClassifier(), 'type': 'tree'}, { 'classifier': RandomForestClassifier(), 'type': 'forest' },
#     #{ 'classifier': GradientBoostingClassifier(), 'type': 'gbc'},
#     #{ 'classifier': XGBClassifier(), 'type': 'xgb'},
#     { 'classifier': AdaBoostClassifier(), 'type': 'ada'},
# ]

# best_params = []

# for tree in tree_classifiers:
#     pipe = make_pipeline(preprocessing, tree['classifier'])
#     rcv = RandomizedSearchCV(pipe, parameters[tree['type']], cv=5, scoring='roc_auc', verbose=True, n_jobs=-1)
    
#     print(f'Fitting: {tree}')
#     rcv.fit(X_train, y_train)
    
#     best_params.append({'clf': tree['classifier'], 'best_params': rcv.best_params_, 'best_score': rcv.best_score_})
#     print(best_params)

# print(f'Best params: {best_params}')

# Переберем разные классификаторы и посмотрим на результат
classifiers = [
    lr0,
    lr1,
    SGDClassifier(loss='log'),
    #KNeighborsClassifier(), - медленная работа и плохой результат
    DecisionTreeClassifier(min_samples_split=2, min_samples_leaf=2, max_features='sqrt', max_depth=10),
    RandomForestClassifier(n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features='auto', max_depth=10, bootstrap=True),
    AdaBoostClassifier(n_estimators=100),
    GradientBoostingClassifier(n_estimators=100),
    XGBClassifier(verbose=False, objective='reg:logistic', n_estimators=100, subsample=1.0, min_child_weight=1, max_depth=4, gamma=5, colsample_bytree=1.0),
    LGBMClassifier(),
    CatBoostClassifier(verbose=False),
    ]

best_score = 0
best_model = 0
best_estimator = []

# Проводим поиск лучшего классификатора
for classifier in classifiers:
    pipe = make_pipeline(preprocessing, classifier)

    print('\n')
    print(f'Fitting {classifier}...')
    pipe.fit(X_train, y_train)
    print(f'Predicting...')
    y_pred = pipe.predict(X_test)
    proba = pipe.predict_proba(X_test)[:, 1]
    
    score = roc_auc_score(y_test, proba)
    print(f'ROC-AUC score is: {score}')
    
    if score > best_score:
        best_score = score
        best_model = classifier
        best_estimator.append([best_score, best_model, pipe])

print(f'Best model is: {best_model} with score: {best_score}')


In [ ]:
val_pipe = make_pipeline(preprocessing, lr1)
validate = cross_validate(val_pipe, X_test, y_test, cv=10, scoring='roc_auc', return_train_score=True)
print(np.mean(validate['test_score']))
val_pipe.fit(X_train, y_train)
plot_confusion_matrix(val_pipe, X_test, y_test, values_format='d')

In [ ]:
get_top_abs_correlations(df.select_dtypes(include=['float64', 'int64']), 10)

Из всех классификаторов лучший результат по метрике roc_auc показывает наша LogisticRegression. Ближе всех по результатам приближаются AdaBoostClassifier и GradientBoostingClassifier. Можно дополнительно попробовать беггинг логистической регрессии раз она показывает лучшие результаты.

Подбор параметров показал, что лучший результат получается, когда в пайплайн добавляем метод главных компонент с количеством признаков 25, но Kaggle говорит нам о другом и результат при использовании PCA падает :(

# Бэггинг логистической регрессии

In [ ]:
from sklearn.ensemble import BaggingClassifier

df = load_data(preprocess=False, drop_outliers=False, outliers_column='income')

# Разобьем данные
X_train, X_test, y_train, y_test = prepare_data(df)

# Посчитаем веса классов
y0 = len(y_train[y_train == 1])
y1 = len(y_train[y_train == 0])

w0 = y1/y0
w1 = 1

sample_weights = np.zeros(len(y_train))
sample_weights[y_train == 0] = w0
sample_weights[y_train == 1] = w1

lr = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1.5, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None, 
                           multi_class= 'auto', 
                           n_jobs= None, 
                           penalty= 'l2', 
                           solver = 'lbfgs',
                           max_iter = 500,
                           verbose= 0, 
                           warm_start= False)

# Предобработка данных
preprocessing = prepare_data_pipeline()

# Модель
model = BaggingClassifier(lr, 
                      n_estimators=5, 
                      bootstrap = True, random_state = RANDOM_SEED)

pipe = make_pipeline(preprocessing, model)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
proba = pipe.predict_proba(X_test)[:, 1]
    
score = roc_auc_score(y_test, proba)

print(f'Model score: {score}')


In [ ]:
val_pipe = make_pipeline(preprocessing, model)
validate = cross_validate(val_pipe, X_test, y_test, cv=10, scoring='roc_auc', return_train_score=True)
print(np.mean(validate['test_score']))
val_pipe.fit(X_train, y_train)
plot_confusion_matrix(val_pipe, X_test, y_test, values_format='d')

Чуда не произошло - ансамбль логистических регрессий результат не улучшил.

# Стэкинг моделей

Попробуем объединить в стэк самые популярные алгоритмы классификации.

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import ExtraTreesClassifier

# Загрузим данные
df = load_data(preprocess=False, drop_outliers=False, outliers_column='income')

# Разобьем данные
X_train, X_test, y_train, y_test = prepare_data(df)

# Определяем базовые модели в стэк
base_models = [
                ('bm1', XGBClassifier(objective='reg:logistic', n_estimators=100, subsample=1.0, min_child_weight=1, max_depth=4, gamma=5, colsample_bytree=1.0)),
                ('bm2', LGBMClassifier()),
                ('bm3', RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini')),
                ('bm4', ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini'))
]

# Решающая модель
blending_model = LogisticRegression(random_state=RANDOM_SEED, 
                                       C=1.5, 
                                       class_weight= 'balanced', 
                                       dual= False, 
                                       fit_intercept= True, 
                                       intercept_scaling= 1, 
                                       l1_ratio= None, 
                                       multi_class= 'auto', 
                                       n_jobs= None, 
                                       penalty= 'l2', 
                                       solver = 'lbfgs',
                                       max_iter = 500,
                                       verbose= 0, 
                                       warm_start= False)

# Стэк
stack_model = StackingClassifier(estimators=base_models, final_estimator=blending_model)

# Предобработка данных
preprocessing = prepare_data_pipeline()

# Наш пайп
final_pipe = make_pipeline(preprocessing, stack_model)

# Обучаем стэк
final_pipe.fit(X_train, y_train)
y_pred = final_pipe.predict(X_test)
proba = final_pipe.predict_proba(X_test)[:, 1]
    
score = roc_auc_score(y_test, proba)

print(f'Model score: {score}')

# Feature Engineering

In [155]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

# Загрузим данные
df = load_data(preprocess=False, drop_outliers=False, outliers_column='income')

# Разобьем данные
X_train, X_test, y_train, y_test = prepare_data(df)

# Предобработка данных
preprocessing = prepare_data_pipeline(scaler='robust')

# Классификатор
clf = LogisticRegression(random_state=RANDOM_SEED, 
                                       C=1.5, 
                                       class_weight= 'balanced', 
                                       dual= False, 
                                       fit_intercept= True, 
                                       intercept_scaling= 1, 
                                       l1_ratio= None, 
                                       multi_class= 'auto', 
                                       n_jobs= None, 
                                       penalty= 'l2', 
                                       solver = 'lbfgs',
                                       max_iter = 1000,
                                       verbose= 0, 
                                       warm_start= False)

#num_features = X_test.select_dtypes(include='float64')
num_features = ['decline_app_cnt', 'bki_request_cnt', 'score_bki', 'income']

pipeline_features = [['mul', 'std']]

# results = []
# # Pipeline
# for pipeline_feature in pipeline_features:
#     pipeline = make_pipeline(FeatureGenerator(feature_list=num_features, primitives=pipeline_feature), preprocessing, RobustScaler(), clf)
#     validate = cross_validate(pipeline, X_train, y_train, cv=3, scoring='roc_auc', return_train_score=True)

#     results.append({'score': np.mean(validate['test_score']), 'feature_param': pipeline_feature})

# print(results)
pipeline = make_pipeline(FeatureGenerator(feature_list=num_features, primitives=pipeline_feature), preprocessing, RobustScaler(), clf)
validate = cross_validate(pipeline, X_train, y_train, cv=3, scoring='roc_auc', return_train_score=True)
print(np.mean(validate['test_score']))


0.7326479068274905


- [{'score': 0.7327340883552927, 'feature_param': ['mul', 'mean']}]
- [{'score': 0.7327220181577214, 'feature_param': ['mul']}]
- [{'score': 0.732755392108973, 'feature_param': ['mul', 'std']}]
- [{'score': 0.7327734423515534, 'feature_param': ['mul', 'std', 'sum']}]
- [{'score': 0.732788267061245, 'feature_param': ['mul', 'std', 'sub']}]



# Submission

In [146]:
df = load_data(preprocess=False, drop_outliers=False, outliers_column='income', log_numerical=True)

train_data = df.query('Train == 1').drop(['Train'], axis=1)
test_data = df.query('Train == 0').drop(['Train'], axis=1)

df_test = pd.read_csv(PATH_to_file+'test.csv')

In [147]:
# Удалим выбросы по income
#train_data = remove_outliers_iqr_column(train_data, 'income')

X_train=train_data.drop(['default'], axis=1)
y_train = train_data.default.values
X_test = test_data.drop(['default'], axis=1)

In [148]:
# Наша лучшая модель
lr = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1.5, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None, 
                           multi_class= 'auto', 
                           n_jobs= None, 
                           penalty= 'l2', 
                           solver = 'lbfgs',
                           max_iter = 500,
                           verbose= 0, 
                           warm_start= False)

# Предобработка данных
preprocessing = prepare_data_pipeline(scaler='robust')

# model = BaggingClassifier(lr, 
#                       n_estimators=15, 
#                       bootstrap = True, random_state = RANDOM_SEED)

# best_pipe = make_pipeline(preprocessing, model)
# best_pipe.fit(X_train, y_train)
# y_pred_prob = best_pipe.predict_proba(X_test)[:,1]
# y_pred = best_pipe.predict(X_test)


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
proba = pipeline.predict_proba(X_test)[:, 1]

In [149]:
submit = pd.DataFrame(test_data.client_id)
submit['default']=proba
submit.to_csv('submission.csv', index=False)


**Вывод:** За время хакатона удалось добиться результата метрики: **0.73766**
Данный датасет показал лучшие результаты на логистической регрессии с подобранными GridSearchCV гиперпараметрами. Что не успел: Проверить автоматический фичаинжениринг через featuretools и найти скрытые инсайты.

**Дополнительно:** После хакатона написал класс генерации признаков FeatureGenerator, который методом подбора примитивов (лучшим было перемножение признаков)показал лучший результат: **0.73799**